## Lab01: Collect and preprocess data

- Name: Nguyen Tuan Thanh
- Student code: 21127166

***
## How to do and submit your assignment

**Work on your assignment**

You will do your assignment directly in this notebook. First, fill in your name and ID at the beginning of the file. In the notebook, fill in places that say:
```python
#TODO
```

During your work, you can print out the result, create more cells to test, or create more functions to handle things. Please note that <font color=red>you are not allowed to delete or modify my code cells</font> (except in the case that mentioned above). Let remove `raise NotImplementedError("not implement")` when running code.

Always press `Ctrl + S` in order to save your work.

**Notes:**

    *  Copy means zero
    *  You have to submit your work on time. No exception
    *  Any questions about your grade are based on the assignment submitted on Moodle
    *  Wrong submission takes you -2 points

**Submission guideline**

When grading your assignment, I will choose `Kernel` - `Restart Kernel & Run All Cells` in order to restart the kernel and run all cells in your notebook. Therefore, you should do that before submitting to ensure that the outputs are all as expected.

After that, you make a submited direction as follow:

- Folder `StudentCode` (e.g. If your student code is 1234567, then your folder is `1234567`)
    - File `<StudentCode>.ipynb` (e.g. If your student code is 1234567, then your file is `1234567.ipynb`)

Finally, you compress your folder (`StudentCode`) and submit on Moodle. **The extension of the file is nothing else but `.zip`.**

<font color=red>Please strictly follow the submission rules.</font>

# 1. Set-up environment

In [21]:
#Necessary Packages
import time
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
# YOUR CODE HERE (OPTION)
# If you need other support packages

# 2. Collect data from a website by parsing HTML (3p)

In this section, you are going to collect data from a website simulating the sale of Pokemon. I have prepared all the needed links in a file (`pokemon.txt`), and you have to crawl data from these links. The expected output is a `dataframe` with the following fields:

+ `SKU`: ID of Pokemon
+ `Name`: Name of Pokemon
+ `Price`: Price of Pokemon
+ `InStock`: Quantity of Pokemons in stock
+ `Categories`: The category of Pokemon
+ `Tags`: Tags of Pokemon

Your mission is to complete function `collect_data` with `course_urls_file` as your input parameter. The output should look like `pokemon_example.csv` (I list out some examples so you can easily imagine your work).

In [ ]:
pokemon_example = pd.read_csv('pokemon_example.csv')
pokemon_example

In [ ]:
pokemon_example.info()

In [ ]:
def collect_data(course_urls_file):
    #load paths from file
    url_file = open(course_urls_file)
    urls = url_file.readlines()
    urls_filtered = [item[:-1] for item in urls]

    #init empty list to store the values of each attribute.
    SKU = []
    Names = []
    Prices = []
    InStocks = []
    Categories = []
    Tags = []

    for url in urls_filtered:
        try:
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            sku = soup.find('span', class_='sku')
            SKU.append(int(soup.find('span', class_='sku').text) if sku else None)
            
            name = soup.find('h1', class_='product_title entry-title')
            Names.append(soup.find('h1', class_='product_title entry-title').text if name else None)

            price = soup.find('p', class_='price') 
            Prices.append(float(price.text.replace('£', '')) if price else None)

            in_stock = soup.find('p', class_='stock in-stock')
            InStocks.append(int(in_stock.text.split()[0]) if in_stock else None)

            category = soup.find('span', class_='posted_in')
            Categories.append(category.text.split(':')[1].strip() if category else None)

            tag = soup.find('span', class_='tagged_as')
            Tags.append(tag.text.split(':')[1].strip() if tag else None)

        except requests.exceptions.RequestException as e:
            print(f"Error processing URL: {url}, Error: {str(e)}")


    data = pd.DataFrame({"SKU": SKU,
                         "Name": Names,
                         "Price": Prices,
                         "InStock": InStocks,
                         "Categories": Categories,
                         "Tags": Tags})

    return data

In [ ]:
#TEST
data_pokemon = collect_data("pokemon.txt")
assert data_pokemon.shape == (755, 6)

In [ ]:
#Save to csv file with name pokemon.csv
data_pokemon.reset_index(drop=True, inplace=True)
# data_pokemon[['SKU', 'InStock']] = data_pokemon[['SKU', 'InStock']].astype('int64')
# data_pokemon['Price'] = data_pokemon['Price'].astype('float64')

data_pokemon.to_csv('pokemon.csv', index=False)
data_pokemon.head()

In [ ]:
data_pokemon.tail()

In this section, your work is to practice to crawl data using Web API (http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL). This is the data of World Bank which includes demographic data and other statistics related to Population, Employment, Health, GDP, Energy Consumption,... for all countries in the world from 1960 to 2022.

From the following selected indicators:
- `SP.POP.TOTL` - Total population
- `SP.POP.TOTL.FE.IN` - Total female population
- `SP.POP.TOTL.MA.IN` - Total male population
- `SP.DYN.CBRT.IN` - Birth rate
- `SP.DYN.CDRT.IN` - Death rate
- `SP.DYN.LE00.MA.IN` - Average life expectancy of male
- `SP.DYN.LE00.FE.IN` - Average life expectancy of female
- `SE.PRM.ENRR` - Primary school enrollment rate
- `SE.TER.ENRR` - High school enrollment rate
- `SE.PRM.CMPT.ZS` - Primary completion rate
- `SE.ADT.1524.LT.ZS` - Literacy rate of people ages 15-24

You are required to collect data from 7 countries and save to dataframe `data_countries`:
- `US` - United States of America
- `IN` - India
- `CN` - China
- `JP` - Japan
- `CA` - Canada
- `GB` - Great Britain
- `ZA` - South Africa

You can expand your work on collecting data (such as collecting data from other countries and other indicators) by reading: https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-api-documentation

**Hints**:

- Use the based URL: http://api.worldbank.org/v2/
- In order to collect data for each indicator of each country, you can use the URL: "http://api.worldbank.org/v2/countries/{country_code}/indicators/{indicator_code}"
    + `country_code` and `indicator_code` are provided above.
    + For example, you can use the following URL to get the `Total population` of Japan: http://api.worldbank.org/v2/countries/jp/indicators/SP.POP.TOTL

# 3. Collect data using Web API (4p)

In [22]:
data_countries_examples = pd.read_csv("countries_example.csv")
data_countries_examples

,Total Population,Female Population,Male Population,Birth Rate,Death Rate,Male life expectancy,Female life expectancy,"School enrollment, primary","School enrollment, tertiary",Primary completion rate,Literacy rate,Year,Country
0,333287557.0,168266219.0,165021339.0,None,None,None,None,None,None,None,None,2022,USA
1,332031554.0,167550001.0,164481553.0,11.0,10.4,73.5,79.3,None,None,None,None,2021,USA
2,331511512.0,167203010.0,164308503.0,10.9,10.3,74.2,79.9,100.305793762207,87.5676574707031,100.923667907715,None,2020,USA
3,328329953.0,165599805.0,162730147.0,11.4,8.7,76.3,81.4,100.981300354004,87.8887100219727,100.489051818848,None,2019,USA
4,326838199.0,164926348.0,161911851.0,11.6,8.678,76.2,81.2,101.256561279297,88.2991790771484,100.092697143555,None,2018,USA
5,325122128.0,164151818.0,160970309.0,11.8,8.638,76.1,81.1,101.821441650391,88.1673889160156,98.8321990966797,None,2017,USA
6,323071755.0,163224028.0,159847727.0,12.2,8.493,76.1,81.1,101.362861633301,88.8350524902344,None,None,2016,USA
7,320738994.0,162158414.0,158580581.0,12.4,8.44,76.3,81.2,100.299911499023,88.8894119262695,None,None,2015,USA
8,318386329.0,161084758.0,157301571.0,12.5,8.237,76.5,81.3,99.6733779907227,88.6268692016602,None,None,2014,USA
9,316059947.0,160034189.0,156025758.0,12.4,8.215,76.4,81.2,99.455436706543,88.7264175415039,None,None,2013,USA


In [23]:
data_countries_examples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Total Population             10 non-null     float64
 1   Female Population            10 non-null     float64
 2   Male Population              10 non-null     float64
 3   Birth Rate                   10 non-null     object 
 4   Death Rate                   10 non-null     object 
 5   Male life expectancy         10 non-null     object 
 6   Female life expectancy       10 non-null     object 
 7   School enrollment, primary   10 non-null     object 
 8   School enrollment, tertiary  10 non-null     object 
 9   Primary completion rate      10 non-null     object 
 10  Literacy rate                10 non-null     object 
 11  Year                         10 non-null     int64  
 12  Country                      10 non-null     object 
dtypes: float64(3), int64(1)

In [24]:
BASE_URL = 'http://api.worldbank.org/v2/'
COUNTRIES = ["US", "IN", "CN", "JP", "CA", "GB", "ZA"]
INDICATORS = ['SP.POP.TOTL',
             'SP.POP.TOTL.FE.IN',
             'SP.POP.TOTL.MA.IN',
             'SP.DYN.CBRT.IN',
             'SP.DYN.CDRT.IN',
             'SP.DYN.LE00.MA.IN',
             'SP.DYN.LE00.FE.IN',
             'SE.PRM.ENRR',
             'SE.TER.ENRR',
             'SE.PRM.CMPT.ZS',
             'SE.ADT.1524.LT.ZS']

INDICATOR_NAMES = [
        'Total Population',
        'Female Population',
        'Male Population',
        'Birth Rate',
        'Death Rate',
        'Male Life Expectancy',
        'Female Life Expectancy',
        'School Enrollment, Primary',
        'School Enrollment, Tertiary',
        'Primary Completion Rate',
        'Literacy Rate']

#TODO (option)
# If you need other initializations

In [25]:
def collect_data(countryCode, per_page, start_year, end_year):
    data_country = {indicator_name: [] for indicator_name in INDICATOR_NAMES}

    for i, indicator in enumerate(INDICATORS):
        api_url = f"{BASE_URL}/countries/{countryCode}/indicators/{indicator}"

        response = requests.get(api_url)
        if response.status_code != 200:
            print(f"Failed to fetch data for {countryCode} - {indicator}")
            continue

        soup = BeautifulSoup(response.content, 'lxml')
        data_elements = soup.find_all('wb:data')[1:]

        values, years = [], []
        for data in data_elements:
            current_year = int(data.find('wb:date').text)
            if start_year <= current_year <= end_year:
                years.append(current_year)
                values.append(data.find('wb:value').text if data.find('wb:value').text != '' else None)

        data_country[INDICATOR_NAMES[i]] = values

    data_country['Year'] = years
    data_country['Country'] = [countryCode] * len(years)

    df = pd.DataFrame(data_country)
    return df


In [26]:
def Generate_Countries_Dataset(countryCode_list):
    data = pd.DataFrame()
    for countryCode in countryCode_list:
        data = pd.concat([data, collect_data(countryCode = countryCode, per_page = 100, start_year = 2000, end_year = 2022)], axis=0)
    return data

In [30]:
#TEST
data_countries = Generate_Countries_Dataset(COUNTRIES)
assert data_countries.shape == (161, 13)

In [31]:
# Save to csv file with name countries.csv
data_countries.reset_index(drop=True, inplace=True)  
data_countries[['Total Population', 'Female Population', 'Male Population']] = data_countries[['Total Population', 'Female Population', 'Male Population']].astype('float64')

data_countries.to_csv('countries.csv', index=False)  
print("Data saved to countries.csv")
data_countries.head()


Data saved to countries.csv


,Total Population,Female Population,Male Population,Birth Rate,Death Rate,Male Life Expectancy,Female Life Expectancy,"School Enrollment, Primary","School Enrollment, Tertiary",Primary Completion Rate,Literacy Rate,Year,Country
0,333287557.0,168266219.0,165021339.0,None,None,None,None,None,None,None,None,2022,US
1,332031554.0,167550001.0,164481553.0,11,10.4,73.5,79.3,None,None,None,None,2021,US
2,331511512.0,167203010.0,164308503.0,10.9,10.3,74.2,79.9,100.305793762207,87.5676574707031,100.923667907715,None,2020,US
3,328329953.0,165599805.0,162730147.0,11.4,8.7,76.3,81.4,100.981300354004,87.8887100219727,100.489051818848,None,2019,US
4,326838199.0,164926348.0,161911851.0,11.6,8.678,76.2,81.2,101.256561279297,88.2991790771484,100.092697143555,None,2018,US


In [32]:
data_countries.tail()

,Total Population,Female Population,Male Population,Birth Rate,Death Rate,Male Life Expectancy,Female Life Expectancy,"School Enrollment, Primary","School Enrollment, Tertiary",Primary Completion Rate,Literacy Rate,Year,Country
156,48556071.0,25379489.0,23176582.0,21.261,12.947,51.891,55.721,106.446632385254,None,95.6549224853516,None,2004,ZA
157,48104048.0,25178665.0,22925383.0,20.473,12.657,51.97,56.232,105.426368713379,None,90.579948425293,None,2003,ZA
158,47661514.0,24979734.0,22681781.0,20.04,11.784,53.191,57.668,103.968452453613,None,86.3408279418945,None,2002,ZA
159,47229714.0,24781908.0,22447806.0,20.392,10.871,54.565,59.47,101.990463256836,None,None,None,2001,ZA
160,46813266.0,24591889.0,22221377.0,20.705,10.174,55.397,61.049,101.489212036133,None,81.0422515869141,None,2000,ZA
